email: `jocaxex742@alibrs.com`
password:`jocaxex742jocaxex742`

In [1]:
import pandas as pd
import re

In [2]:
## Parsing
# Documents
with open('TIME.ALL', 'r') as f:
    text = f.read()
result = re.findall(r"\*TEXT\s+(\d{3})\s+(\d{2}/\d{2}(/|\s)\d{2})\s+PAGE\s+(\d{3})\n\n(.+?)(?=\*TEXT|$)", text, re.DOTALL)
docDB = {match[0]: {'id': match[0], 'date': match[1], 'page': int(match[3]), 'text': match[4]} for match in result}
docDF = pd.DataFrame(docDB).T

# Queries
with open('TIME.QUE', 'r') as f:
    text = f.read()
queryDB = re.findall(r'FIND\s+\d+\s+(.*)', text)

# Stopwords
with open('TIME.STP','r') as f:
    text = f.read()
swDB = re.findall(r"^[A-Z]+$", text, re.MULTILINE)

# Relevant docs
with open('TIME.REL', 'r') as f:
    text = f.read()
    lines = text.split("\n")
rdDB = {}
for line in lines:
    numbers = re.findall(r"\d+", line)
    if numbers:
        key = numbers[0]
        values = [int(n) for n in numbers[1:]]
        rdDB[key] = values

# Elasticsearch

In [3]:
from datetime import datetime
from elasticsearch import Elasticsearch

client = Elasticsearch('https://jocaxex.es.ap-south-1.aws.elastic-cloud.com', api_key='amVjQVlJMEJrU21yN1VtV0Jsd1I6cGxVSVdXOWJTQnk5ODc3dWozVWZHQQ==')

In [4]:
# Create index
client.indices.create(index='lol')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'lol'})

In [6]:
# Indexing the DB
indexingResponses = [client.index(index='lol', id=docID, document=doc) for docID, doc in docDB.items()]
indexingResponses

[ObjectApiResponse({'_index': 'lol', '_id': '017', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}),
 ObjectApiResponse({'_index': 'lol', '_id': '018', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}),
 ObjectApiResponse({'_index': 'lol', '_id': '019', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}),
 ObjectApiResponse({'_index': 'lol', '_id': '020', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 3, '_primary_term': 1}),
 ObjectApiResponse({'_index': 'lol', '_id': '021', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 4, '_primary_term': 1}),
 ObjectApiResponse({'_index': 'lol', '_id': '023', '_version': 1, 'result': 'created', '_shards': {'total': 2,

In [30]:
queryResponseMap = {str(queryIdx + 1):[hit['_id'] for hit in dict(client.search(index='lol', query={
    "match": {
        "text": queryText
    }
}))['hits']['hits']] for queryIdx, queryText in enumerate(queryDB)}
queryResponseMap

{'1': ['464', '544', '390', '498', '363', '334', '320', '508', '396', '414'],
 '2': ['414', '518', '390', '498', '418', '464', '434', '396', '508', '545'],
 '3': ['060', '027', '253', '223', '464', '295', '029', '563', '024', '396'],
 '4': ['518', '269', '396', '418', '334', '434', '414', '464', '545', '559'],
 '5': ['518', '533', '434', '508', '504', '390', '418', '464', '498', '480'],
 '6': ['334', '418', '414', '320', '533', '396', '415', '363', '434', '390'],
 '7': ['544', '561', '519', '081', '300', '122', '201', '101', '226', '063'],
 '8': ['479', '443', '094', '203', '335', '204', '303', '389', '270', '265'],
 '9': ['479', '443', '335', '204', '094', '303', '203', '562', '389', '321'],
 '10': ['094', '335', '203', '543', '081', '444', '226', '519', '402', '470'],
 '11': ['183', '256', '299', '196', '335', '326', '094', '204', '443', '070'],
 '12': ['094', '335', '303', '204', '203', '479', '299', '054', '444', '256'],
 '13': ['361', '227', '300', '446', '243', '280', '226', '149